# Grouping and resampling

xarray was developed as an n-dimensional extension of [pandas](https://pandas.pydata.org), a very powerful data analysis library designed primarily for tabular data and time series analysis.

As a result xarray can utilise much of the time series manipulation power of pandas. The relevant xarray documentation is contained in the [GroupBy](http://xarray.pydata.org/en/stable/groupby.html) and [Time series data](http://xarray.pydata.org/en/stable/time-series.html) sections.

In this case we need to do some extra imports to avoid pandas emitting warnings

In [ ]:
import xarray, matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
%matplotlib inline

Load a CMIP5 air temperature data set into the variable `ds`, and make a variable, `tas`, which directly references the near surface air temperature `DataArray`

In [ ]:
ds = xarray.open_dataset('http://dapds00.nci.org.au/thredds/dodsC/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/mon/atmos/Amon/r1i1p1/latest/tas/tas_Amon_ACCESS1-3_historical_r1i1p1_185001-200512.nc')
tas = ds.tas

Calculate the 30 year climatology and anomalies as in the previous tutorial

In [ ]:
tas_clim = tas.sel(time=slice('1960-01','1989-12')).mean(dim='time')
tas_anom = (tas - tas_clim)

As before, plotting the global anomaly from 1960 by time

In [ ]:
tas_anom.sel(time=slice('1960-01',None)).mean(dim=('lat','lon')).plot(size=6)

To highlight any global warming trend it would be better to show annual values. This can be done by resampling the data to an annual frequency. The `resample` method takes a dimension and resample period key/pair. In this case yearly frequency is specified with `Y` (see the pandas documentation for a [complete listing of frequency strings](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects)).

The `resample` method returns a resample object, which could be iterated over. In this case by specifying an operator (`mean`) this will be applied to each resampled period in turn, producing an annual mean in this case. Note that it is necessary to specify a dimension over which to apply the `mean` operator, otherwise  the operator will be applied across `lat` and `lon` reducing the dimensionality of the result. This is ultimately what is required, but keeping the spatial dimensions at this point is useful

In [ ]:
tas_anom_yearly = tas_anom.sel(time=slice('1960-01',None)).resample(time='Y').mean(dim='time')
tas_anom_yearly.dims, tas_anom_yearly.shape

The dimensions and shape show the time dimension has been reduced to 46 years

Now the global `mean` shows a clear warming trend

In [ ]:
tas_anom_yearly.mean(dim=('lat','lon')).plot(size=6)

Because `tas_anom_yearly` still has spatial dimensions it is now possible to use `.sel` to calculate the same trend for the northern and southern hemisphere separately.

In the plot below to make the lines appear on the same plot it is necessary to specify the `matplotlib.axes` object directly so that it can be passed to each of the plots

In [ ]:
axes = plt.axes()
axes.figure.set_size_inches(8, 6)
tas_anom_yearly.mean(dim=('lat','lon')).plot(ax=axes, label='Global')
tas_anom_yearly.sel(lat=slice(0,None)).mean(dim=('lat','lon')).plot(ax=axes, label='Northern Hemisphere')
tas_anom_yearly.sel(lat=slice(None,0)).mean(dim=('lat','lon')).plot(ax=axes, label='Southern Hemisphere')
plt.title('Temperature anomalies from a 1960-1990 climatology')
plt.legend()

Splitting into hemispheres shows both hemispheres are in sync for most of the plotted interval, but diverge in the last few years.

To go further and split the trend more finely in latitude it is easier to use the `xarray.groupby_bins` function. It is called with a coordinate and either a number of `bins` or an array of values specifying the start and end of each bin.

It returns a `groupby` object which can be iterated over, or chain a function which will applied to each group. 

To understand what the `groupby` is doing, here the `for` loop is iterating over the `groupby` object, which returns a tuple of interval and `DataArray` object. The interval and the dimensions and their shape show that each is just a `slice` of the data in latitude.

In [ ]:
for i, d in tas_anom_yearly.groupby_bins('lat', bins=10):
    print(i, d.dims, d.shape)

Below the same `groupby_bins` operation has the `mean` in `lat` and `lon` applied and then plotted. The `lat_bins` dimension is created by xarray, and by specifying that as the `hue` xarray automatially generates 10 individual lines, one for each bin, on the same plot.

In [ ]:
tas_anom_yearly.groupby_bins('lat', bins=10).mean(dim=('lat','lon')).plot(x='time',hue='lat_bins',size=8)

Finely grouping in latitude shows the rapid increase in anomalies at the end of the time series is dominated by large changes at the northern high latitudes.

xarray has some special virtual time coordinates. `time.season` is one of these. Below the seasonal mean of `tas` for each location is calculated and all plotted in a `FacetGrid` by specifying a `col` coordinate option to `plot`

In [ ]:
tas.groupby('time.season').mean(dim='time').plot(col='season', col_wrap=2)